<div class="alert alert-danger" role="alert">

**IN THIS NOTEBOOK WE ANALYZE AND PLOT TRANSFERS OF SIZE 50-75 MILLION EUROS FOR DIFFERENT YEARS**
    
</div>

**PREAMBLE**

In [50]:
import pandas as pd
#import fuzzy_pandas as fpd
import numpy as np

import plotly.graph_objects as go
import plotly.express as px
import plotly

#for offline plotting using plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

**CLEANING STADIUM COORDINATES**

In [51]:
stadia = pd.read_csv("Data/Stadium_Coordinates_EDA.csv")

stadia['coordinates'] = stadia['coordinates'].str.replace("Point\(","") 
stadia['coordinates'] = stadia['coordinates'].str.replace("\)","") 

temp = stadia['coordinates'].str.split(expand = True)

temp = temp.rename(columns={0:'Longitude',1:'Latitude'})

a = temp[['Latitude','Longitude']].astype(float)

stadia = stadia.join(a)
stadia_cleaned = stadia.drop_duplicates(subset=['venueLabel'], keep="first", inplace=False)

stadia_cleaned = stadia_cleaned[['clubLabel','venueLabel','Latitude','Longitude']]

stadia_cleaned

,clubLabel,venueLabel,Latitude,Longitude
0,S.S.C. Napoli,Stadio San Paolo,40.827967,14.193008
2,Manchester United F.C.,Old Trafford,53.463056,-2.291389
3,SD Compostela,Estadio Multiusos de San Lázaro,42.882767,-8.516014
6,Sacramento Knights,Sleep Train Arena,38.649167,-121.518056
7,Novara Calcio,Stadio Silvio Piola (Novara),45.435833,8.595833
...,...,...,...,...
9035,FK Smiltene/BJSS,Teperis stadium,57.430749,25.920292
9036,Pyrgos Limassol Women FC,Pyrgos Community Stadium,34.746644,33.188163
9037,Juventude Sport Clube,Sanches de Miranda Stadium,38.561944,-7.913889
9038,Louisville City FC,Lynn Family Stadium,38.259590,-85.733264


**TRANSFER MARKT DATASET**

In [52]:
#Test Dataset
Test_1 = pd.read_csv("Data/transfer_data_Final_Master_2010_2011_Jul.csv")
Test_2 = pd.read_csv("Data/transfer_data_Final_Master_2011_Aug_2012_Jun.csv")
Test_3 = pd.read_csv("Data/transfer_data_Final_Master_2012_Jul_2015.csv")
Test_4 = pd.read_csv("Data/transfer_data_Final_Master_2016_Aug_Dec.csv")
Test_5 = pd.read_csv("Data/transfer_data_Final_Master_2016_Jan_Jul.csv")
Test_6 = pd.read_csv("Data/transfer_data_Final_Master_2017.csv")
Test_7 = pd.read_csv("Data/transfer_data_Final_Master_2018.csv")
Test_8 = pd.read_csv("Data/transfer_data_Final_Master_2019.csv")
Test_9 = pd.read_csv("Data/transfer_data_Final_Master_2005.csv")
Test_10 = pd.read_csv("Data/transfer_data_Final_Master_2006.csv")
Test_11 = pd.read_csv("Data/transfer_data_Final_Master_2008.csv")
Test_12 = pd.read_csv("Data/transfer_data_Final_Master_2009.csv")
Test_13 = pd.read_csv("Data/transfer_data_Final_Master_2007.csv")
Test_14 = pd.read_csv("Data/transfer_data_Final_Master_2007_Apr.csv")
Test_EDA = pd.concat([Test_1, Test_2, Test_3,
                      Test_4, Test_5, Test_6, 
                      Test_7, Test_8, Test_9, 
                      Test_10, Test_11, Test_12, 
                      Test_13, Test_14], ignore_index=True)

# Cleaning the name for the column Club left
New = Test_EDA['Club Left'].str.replace('\n', '')
New = New.to_frame(name = 'Club_Left')

# Cleaning the name for the column Club Joined
New1 = Test_EDA['Club Joined'].str.replace('\n', '')
New1 = New1.to_frame(name = 'Club_Joined')

# Cleanning Transfer Data
New2 = Test_EDA['Transfer'].str.replace('?', '0')
New2 = New2.to_frame(name = 'Transfer_Fee')

New2["Transfer_Fee"]=New2["Transfer_Fee"].str.replace("Free transfer","0")
New2["Transfer_Fee"]=New2["Transfer_Fee"].str.replace("-","0")
New2["Transfer_Fee"]=New2["Transfer_Fee"].str.replace(",","")
New2["Transfer_Fee"]=New2["Transfer_Fee"].str.replace("draft","0")
New2["Transfer_Fee"]=New2["Transfer_Fee"].str.replace("Draft","0")
New2["Transfer_Fee"]=New2["Transfer_Fee"].str.replace(" mil. €","0000")
New2["Transfer_Fee"]=New2["Transfer_Fee"].str.replace(" K €","000")
New2["Transfer_Fee"]=New2["Transfer_Fee"].str.replace(" €","")

# Cleaning Market Valuation data
New3 = Test_EDA['Market Valuation'].str.replace(',', '')
New3 = New3.to_frame(name = 'Market_Valuation')
New3["Market_Valuation"]=New3["Market_Valuation"].str.replace(" mil. €","0000")
New3["Market_Valuation"]=New3["Market_Valuation"].str.replace(" K €","000")

#Replacing with original data
Test_EDA["Club Left"]=New["Club_Left"]
Test_EDA["Club Joined"]=New1["Club_Joined"]
Test_EDA["Transfer"]=New2["Transfer_Fee"]
Test_EDA["Market Valuation"]=New3["Market_Valuation"]

# Converting the Date from object ot a datetime format
Test_EDA['Date'] = pd.to_datetime(Test_EDA['Date'], errors='coerce')

# Removing retired players
Test_EDA.rename(columns={"Club Joined": "Club_Joined"}, inplace=True)
Test_EDA=Test_EDA[~Test_EDA.Club_Joined.str.contains("Retired")]
Test_EDA.rename(columns={"Club_Joined": "Club Joined"}, inplace=True)

Test_EDA.reset_index(inplace=True)

#Converting to float
Test_EDA["Transfer"]=Test_EDA['Transfer'].apply(lambda x:float(x))
Test_EDA["Market Valuation"]=Test_EDA["Market Valuation"].apply(lambda x:float(x))
Test_EDA["Age"]=Test_EDA['Age'].apply(lambda x:float(x))

#Adding surplus/deficit column
Test_EDA.rename(columns={"Market Valuation": "Market_Valuation"}, inplace=True)
Test_EDA["Surplus/Deficit"]=Test_EDA.Transfer-Test_EDA.Market_Valuation
Test_EDA.rename(columns={"Market_Valuation": "Market Valuation"}, inplace=True)

#Adding year column
Test_EDA["Year"]=pd.DatetimeIndex(Test_EDA["Date"]).year

Test_EDA

,index,Date,Name,Age,Position,Nationality,Club Left,League Left,Club Joined,League Joined,Market Valuation,Transfer,Surplus/Deficit,Year
0,0,2010-01-01,Douglas Costa,19.0,Right Winger,Brazil,Grêmio,Série A,Shakhtar D.,NaN,4000000.0,8000000.0,4000000.0,2010
1,1,2010-01-01,Florent Sinama-Pongolle,25.0,Second Striker,France,Atlético Madrid,LaLiga,Sporting CP,NaN,7000000.0,6500000.0,-500000.0,2010
2,2,2010-01-01,Alex Teixeira,19.0,Left Winger,Brazil,Vasco da Gama,Série A,Shakhtar D.,NaN,3500000.0,6000000.0,2500000.0,2010
3,3,2010-01-01,Keisuke Honda,23.0,Attacking Midfield,Japan,VVV-Venlo,Eredivisie,CSKA Moscow,NaN,3000000.0,6000000.0,3000000.0,2010
4,4,2010-01-01,Younès Kaboul,23.0,Centre-Back,France,Portsmouth,Premier League,Spurs,NaN,5700000.0,5900000.0,200000.0,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61835,63255,2007-09-25,Thomas Sowunmi,29.0,Centre-Forward,Hungary,Without Club,NaN,Vasas FC,NaN,300000.0,0.0,-300000.0,2007
61836,63256,2007-09-28,Onur Karali,25.0,Midfielder,Turkey,Osmaniyespor,Turkey\t\t,Altinova Bld,NaN,200000.0,0.0,-200000.0,2007
61837,63257,2007-09-28,Enver Isik,22.0,Right Winger,Turkey,Kayserispor,Süper Lig,Göztepe,NaN,250000.0,0.0,-250000.0,2007
61838,63258,2007-09-28,Mustafa Özzengi,20.0,Goalkeeper,Turkey,Galatasaray U21,Turkey\t\t,Bakirköyspor,NaN,50000.0,0.0,-50000.0,2007


**EXTRACTING LIST OF CLUBS IN THE TRANSFER MARKT DATASET**

In [53]:
Test_EDA[['Club Left','Club Joined']]

clubs_left_names = Test_EDA['Club Left'].unique()
clubs_joined_names =Test_EDA['Club Joined'].unique()

clubs_left_names = clubs_left_names.tolist()
clubs_joined_names = clubs_joined_names.tolist()

transfer_markt_club_names = np.unique(clubs_left_names + clubs_joined_names)

transfer_markt_club_names = transfer_markt_club_names.tolist()

df_club_names = pd.Series(transfer_markt_club_names)
df_club_names = df_club_names.to_frame()
df_club_names = df_club_names.rename(columns={0:'Club Name'})
df_club_names

,Club Name
0,FK Uzhgorod
1,Gharb Rangers
2,Goyang Zaicro
3,Montello
4,PJ Rangers
...,...
11335,Ümraniyespor
11336,Ünye 1957
11337,Ünyespor
11338,Łagiewniki


**FUZZY MATCHING**

In [36]:
merged_df = pd.read_csv("Data/club_stadium_coordinates_Final.csv")

**CREATING THE MERGED MASTER DATASETS**

In [37]:
final_DataFrame_1 = pd.merge(Test_EDA,merged_df,
                    how='inner',
                    left_on = 'Club Left',
                    right_on='Club Name TM')

final_DataFrame_2 = pd.merge(final_DataFrame_1, merged_df,
                    how='inner',
                    left_on = 'Club Joined',
                    right_on='Club Name TM',
                    suffixes = ('_Left','_Joined'))

final_DataFrame_2 = final_DataFrame_2.rename(columns={'Club Name TM_Joined':'Club_Name_TM_Joined', 
                                                      'Club Name TM_Left':'Club_Name_TM_Left',
                                                      'Club Name Wikidata_Joined':'Club_Name_Wikidata_Joined',
                                                      'Club Name Wikidata_Left':'Club_Name_Wikidata_Left'})

#final_DataFrame_2 = final_DataFrame_2[final_DataFrame_2.Nationality=='China']

final_DataFrame_2

,index,Date,Name,Age,Position,Nationality,Club Left,League Left,Club Joined,League Joined,...,Club_Name_TM_Left,Club_Name_Wikidata_Left,venueLabel_Left,Latitude_Left,Longitude_Left,Club_Name_TM_Joined,Club_Name_Wikidata_Joined,venueLabel_Joined,Latitude_Joined,Longitude_Joined
0,766,2010-03-10,Aleksandr Marenich,20.0,Left Winger,Russia,FK Moskau,Russia\t\t,Spartak V.,NaN,...,FK Moskau,FK Khazar Lankaran,Lankaran City Stadium,38.747189,48.857131,Spartak V.,K.V.C. Westerlo,Het Kuipje,51.094722,4.928333
1,10833,2013-01-23,Dmitri Tikhiy,20.0,Right-Back,Russia,Luch,PFL Ost,Spartak V.,NaN,...,Luch,Deportivo Toluca F.C.,Nemesio Díez Stadium,19.287286,-99.666794,Spartak V.,K.V.C. Westerlo,Het Kuipje,51.094722,4.928333
2,518,2010-02-01,Ivan Stoyanov,26.0,Right Winger,Bulgaria,CSKA Sofia,Friendlies,Spartak V.,NaN,...,CSKA Sofia,PFC CSKA Sofia,Balgarska Armia Stadium,42.684258,23.339769,Spartak V.,K.V.C. Westerlo,Het Kuipje,51.094722,4.928333
3,698,2010-03-01,Abdoul Gafar Mamah,24.0,Right-Back,Togo,Sheriff,Divizia Nationala,Spartak V.,NaN,...,Sheriff,FC Sheriff Tiraspol,Sheriff Stadium,46.838333,29.557500,Spartak V.,K.V.C. Westerlo,Het Kuipje,51.094722,4.928333
4,3009,2011-01-19,Dmitri Grachev,27.0,Centre-Back,Russia,Saturn,Russia\t\t,Spartak V.,NaN,...,Saturn,FC Saturn Ramenskoye,Saturn Stadium,55.577222,38.226944,Spartak V.,K.V.C. Westerlo,Het Kuipje,51.094722,4.928333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2505,38178,2017-08-01,Andrei Sintean,18.0,Centre-Forward,Romania,Poli Timisoara,Liga 1,Slavia Prague B,NaN,...,Poli Timisoara,FC Politehnica Timișoara,Stadionul Dan Păltinișanu,45.740556,21.244167,Slavia Prague B,SK Slavia Prague,Sinobo Stadium,50.066944,14.471389
2506,49896,2019-03-03,Jamshid Khasanov,30.0,Attacking Midfield,Uzbekistan,AGMG Olmaliq,Superliga,Kokand 1912,NaN,...,AGMG Olmaliq,Nîmes Olympique,Stade des Costières,43.815972,4.359278,Kokand 1912,Békéscsaba 1912 Előre SE,Stadion Kórház utcai,46.686111,21.105000
2507,30578,2016-12-31,Liangxuan Gong,23.0,Defender,China,SC Longfor,League Two South,Evergrande Res.,NaN,...,SC Longfor,Orlando City SC,Camping World Stadium,28.539167,-81.402778,Evergrande Res.,Guangzhou Evergrande Taobao Football Club,Tianhe Stadium,23.138031,113.324661
2508,30578,2016-12-31,Liangxuan Gong,23.0,Defender,China,SC Longfor,League Two South,Evergrande Res.,NaN,...,SC Longfor,Orlando City SC,ESPN Wide World of Sports Complex,28.337083,-81.556000,Evergrande Res.,Guangzhou Evergrande Taobao Football Club,Tianhe Stadium,23.138031,113.324661


**CREATING THE DATA SET FOR THE NODES**

In [38]:
merged_df

Transfer_Value = final_DataFrame_2.groupby('Club Joined').agg({'Transfer':'sum'}).reset_index()

Transfer_Surplus = final_DataFrame_2.groupby('Club Joined').agg({'Surplus/Deficit':'sum'})

merged_df_x = pd.merge(Transfer_Surplus,Transfer_Value,
                    how='outer',
                    on = 'Club Joined')

merged_df_2 = pd.merge(merged_df,merged_df_x,
                    how='inner',
                    left_on = 'Club Name TM',
                    right_on = 'Club Joined')

merged_df_2 = merged_df_2.fillna(0)

merged_df_2['Transfer'] = merged_df_2['Transfer']

merged_df_2 

,Club Name TM,Club Name Wikidata,venueLabel,Latitude,Longitude,Club Joined,Surplus/Deficit,Transfer
0,1.FC Heidenheim,1. FC Heidenheim,Voith-Arena,48.668500,10.139300,1.FC Heidenheim,-775000.0,2400000.0
1,1.FC K'lautern,Cusco FC,Estadio Garcilaso,-13.525000,-71.966000,1.FC K'lautern,-2350000.0,4550000.0
2,FC Pune City,Cusco FC,Estadio Garcilaso,-13.525000,-71.966000,FC Pune City,-3600000.0,0.0
3,Perugia,A.C. Perugia Calcio,Stadio Renato Curi,43.106111,12.357222,Perugia,-2200000.0,2200000.0
4,AC Ajaccio,A.C. Ajaccio,Stade François Coty,41.930761,8.776444,AC Ajaccio,-6150000.0,200000.0
...,...,...,...,...,...,...,...,...
371,YB Funde,Sudet,Kouvolan keskuskenttä,60.871944,26.713333,YB Funde,-1500000.0,0.0
372,YC Helanshan,Cheonan FC,Cheongju Sports Complex,36.637851,127.472375,YC Helanshan,-125000.0,0.0
373,Zawisza,Zawisza Bydgoszcz,Zdzisław Krzyszkowiak Stadium,53.145556,18.020556,Zawisza,-1200000.0,0.0
374,Zulte Waregem,S.V. Zulte Waregem,Regenboogstadion,50.883056,3.428889,Zulte Waregem,-2075000.0,1500000.0


**FINAL DATAFRAME FOR THE EDGES**

In [39]:
final_DataFrame_2 = pd.merge(final_DataFrame_2,Transfer_Value,
                    how='inner',
                    on = 'Club Joined')
final_DataFrame_2

,index,Date,Name,Age,Position,Nationality,Club Left,League Left,Club Joined,League Joined,...,Club_Name_Wikidata_Left,venueLabel_Left,Latitude_Left,Longitude_Left,Club_Name_TM_Joined,Club_Name_Wikidata_Joined,venueLabel_Joined,Latitude_Joined,Longitude_Joined,Transfer_y
0,766,2010-03-10,Aleksandr Marenich,20.0,Left Winger,Russia,FK Moskau,Russia\t\t,Spartak V.,NaN,...,FK Khazar Lankaran,Lankaran City Stadium,38.747189,48.857131,Spartak V.,K.V.C. Westerlo,Het Kuipje,51.094722,4.928333,2350000.0
1,10833,2013-01-23,Dmitri Tikhiy,20.0,Right-Back,Russia,Luch,PFL Ost,Spartak V.,NaN,...,Deportivo Toluca F.C.,Nemesio Díez Stadium,19.287286,-99.666794,Spartak V.,K.V.C. Westerlo,Het Kuipje,51.094722,4.928333,2350000.0
2,518,2010-02-01,Ivan Stoyanov,26.0,Right Winger,Bulgaria,CSKA Sofia,Friendlies,Spartak V.,NaN,...,PFC CSKA Sofia,Balgarska Armia Stadium,42.684258,23.339769,Spartak V.,K.V.C. Westerlo,Het Kuipje,51.094722,4.928333,2350000.0
3,698,2010-03-01,Abdoul Gafar Mamah,24.0,Right-Back,Togo,Sheriff,Divizia Nationala,Spartak V.,NaN,...,FC Sheriff Tiraspol,Sheriff Stadium,46.838333,29.557500,Spartak V.,K.V.C. Westerlo,Het Kuipje,51.094722,4.928333,2350000.0
4,3009,2011-01-19,Dmitri Grachev,27.0,Centre-Back,Russia,Saturn,Russia\t\t,Spartak V.,NaN,...,FC Saturn Ramenskoye,Saturn Stadium,55.577222,38.226944,Spartak V.,K.V.C. Westerlo,Het Kuipje,51.094722,4.928333,2350000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2505,38178,2017-08-01,Andrei Sintean,18.0,Centre-Forward,Romania,Poli Timisoara,Liga 1,Slavia Prague B,NaN,...,FC Politehnica Timișoara,Stadionul Dan Păltinișanu,45.740556,21.244167,Slavia Prague B,SK Slavia Prague,Sinobo Stadium,50.066944,14.471389,25000.0
2506,49896,2019-03-03,Jamshid Khasanov,30.0,Attacking Midfield,Uzbekistan,AGMG Olmaliq,Superliga,Kokand 1912,NaN,...,Nîmes Olympique,Stade des Costières,43.815972,4.359278,Kokand 1912,Békéscsaba 1912 Előre SE,Stadion Kórház utcai,46.686111,21.105000,0.0
2507,30578,2016-12-31,Liangxuan Gong,23.0,Defender,China,SC Longfor,League Two South,Evergrande Res.,NaN,...,Orlando City SC,Camping World Stadium,28.539167,-81.402778,Evergrande Res.,Guangzhou Evergrande Taobao Football Club,Tianhe Stadium,23.138031,113.324661,0.0
2508,30578,2016-12-31,Liangxuan Gong,23.0,Defender,China,SC Longfor,League Two South,Evergrande Res.,NaN,...,Orlando City SC,ESPN Wide World of Sports Complex,28.337083,-81.556000,Evergrande Res.,Guangzhou Evergrande Taobao Football Club,Tianhe Stadium,23.138031,113.324661,0.0


**GRAPH RENDERING**

**IN THE ANALYSIS THAT FOLLOWS WE RESTRICT OUR FOCUS BIG TRANSFERS WHICH RANGE BETWEEN 50 MILLION EUROS AND 75 MILLION EUROS**

In [40]:
final_DataFrame_2=final_DataFrame_2[(final_DataFrame_2['Transfer_y']>50000000) & (final_DataFrame_2['Transfer_y']<=75000000)]
final_DataFrame_2.rename(columns={"Transfer_y": "Transfer"}, inplace=True)
final_DataFrame_2

,index,Date,Name,Age,Position,Nationality,Club Left,League Left,Club Joined,League Joined,...,Club_Name_Wikidata_Left,venueLabel_Left,Latitude_Left,Longitude_Left,Club_Name_TM_Joined,Club_Name_Wikidata_Joined,venueLabel_Joined,Latitude_Joined,Longitude_Joined,Transfer


**FIRST WE LOOK AT YEARS BETWEEN 2005 AND 2010**

In [41]:
years = [2005, 2010]

final_DataFrame_2005_2010 = final_DataFrame_2[final_DataFrame_2.Year.isin(years)]
final_DataFrame_2005_2010

,index,Date,Name,Age,Position,Nationality,Club Left,League Left,Club Joined,League Joined,...,Club_Name_Wikidata_Left,venueLabel_Left,Latitude_Left,Longitude_Left,Club_Name_TM_Joined,Club_Name_Wikidata_Joined,venueLabel_Joined,Latitude_Joined,Longitude_Joined,Transfer


**CREATING THE DATASET FOR THE NODES FOR YEARS 2005-2010**

In [42]:
merged_df

Transfer_Value_2005_2010 = final_DataFrame_2005_2010.groupby('Club Joined').agg({'Transfer':'sum'}).reset_index()

Transfer_Surplus_2005_2010 = final_DataFrame_2005_2010.groupby('Club Joined').agg({'Surplus/Deficit':'sum'})

merged_df_2005_2010_x = pd.merge(Transfer_Surplus_2005_2010,Transfer_Value_2005_2010,
                    how='outer',
                    on = 'Club Joined')

merged_df_2005_2010 = pd.merge(merged_df,merged_df_2005_2010_x,
                    how='inner',
                    left_on = 'Club Name TM',
                    right_on = 'Club Joined')

merged_df_2005_2010 = merged_df_2005_2010.fillna(0)

merged_df_2005_2010['Transfer'] = merged_df_2005_2010['Transfer']

merged_df_2005_2010

,Club Name TM,Club Name Wikidata,venueLabel,Latitude,Longitude,Surplus/Deficit,Club Joined,Transfer


**MASTER DATASET FOR YEARS 2005-2010**

In [43]:
final_DataFrame_2005_2010 = pd.merge(final_DataFrame_2005_2010,Transfer_Value_2005_2010,
                    how='inner',
                    on = 'Club Joined')
final_DataFrame_2005_2010

,index,Date,Name,Age,Position,Nationality,Club Left,League Left,League Joined,Market Valuation,...,Latitude_Left,Longitude_Left,Club_Name_TM_Joined,Club_Name_Wikidata_Joined,venueLabel_Joined,Latitude_Joined,Longitude_Joined,Transfer_x,Club Joined,Transfer_y


<div class="alert alert-warning" role="alert">

**Method** <br>
In following graph, we give a pictorial representation of transfers of size 50-75 million euros in the years 2005-2010 <br><br>

</div>

<div class="alert alert-success" role="alert">
    
**Inference**<br>
    In the five year period of our analysis, we see that the number of 'Big' transfers as defined in the range of 20-75 million euros were quite less in number and as expected were restricted mostly to the European leagues and clubs.

    
</div>

In [44]:
club_left_lat = final_DataFrame_2005_2010.Latitude_Left
club_left_lon = final_DataFrame_2005_2010.Longitude_Left

club_join_lat = final_DataFrame_2005_2010.Latitude_Joined
club_join_lon = final_DataFrame_2005_2010.Longitude_Joined

fig = go.Figure()

fig.add_trace(
        go.Scattergeo(
            lon = merged_df_2005_2010.Longitude,
            lat = merged_df_2005_2010.Latitude,
            text = merged_df_2005_2010['Club Name Wikidata'],
            hoverinfo = 'text',
            mode = 'markers',
            marker = dict(
                size = np.log((merged_df_2005_2010['Transfer']+100)*100000)*0.9,
                opacity = 0.8,
                reversescale = False,
                autocolorscale = False,
                symbol = 'circle',
                line = dict(
                    width=1,
                    color='rgba(102, 102, 102)'),
            colorscale = 'Blues',
            cmin = merged_df_2005_2010['Surplus/Deficit'].min(),
            color = merged_df_2005_2010['Surplus/Deficit'],
            cmax = merged_df_2005_2010['Surplus/Deficit'].max(),
            colorbar_title="Surplus for the Club"
        )))

for i in range(len(final_DataFrame_2005_2010)):
    fig.add_trace(
        go.Scattergeo(
            lon = [final_DataFrame_2005_2010['Longitude_Left'][i], final_DataFrame_2005_2010['Longitude_Joined'][i]],
            lat = [final_DataFrame_2005_2010['Latitude_Left'][i], final_DataFrame_2005_2010['Latitude_Joined'][i]],
            mode = 'lines',
            hovertext = "Name: " + str(final_DataFrame_2005_2010['Name'][i]),
            line = dict(
                width = 1,
                color='darkgray')
        )
    )
    
fig.update_layout(
    title='Transfer Size: 50M to 75M (2005-2010)',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    mapbox_style="open-street-map"
    )

plot(fig, filename = 'Transfer_Size_50M_75M_2005_2010.html')

'Transfer_Size_50M_75M_2005_2010.html'

**NOW WE LOOK AT YEARS 2011-2015**

In [45]:
years = [2011, 2015]

final_DataFrame_2011_2015 = final_DataFrame_2[final_DataFrame_2.Year.isin(years)]

merged_df

Transfer_Value_2011_2015 = final_DataFrame_2011_2015.groupby('Club Joined').agg({'Transfer':'sum'}).reset_index()

Transfer_Surplus_2011_2015 = final_DataFrame_2011_2015.groupby('Club Joined').agg({'Surplus/Deficit':'sum'})

merged_df_2011_2015_x = pd.merge(Transfer_Surplus_2011_2015,Transfer_Value_2011_2015,
                    how='outer',
                    on = 'Club Joined')

merged_df_2011_2015 = pd.merge(merged_df,merged_df_2011_2015_x,
                    how='inner',
                    left_on = 'Club Name TM',
                    right_on = 'Club Joined')

merged_df_2011_2015 = merged_df_2011_2015.fillna(0)

merged_df_2011_2015['Transfer'] = merged_df_2011_2015['Transfer']

final_DataFrame_2011_2015 = pd.merge(final_DataFrame_2011_2015,Transfer_Value_2011_2015,
                    how='inner',
                    on = 'Club Joined')
final_DataFrame_2011_2015

,index,Date,Name,Age,Position,Nationality,Club Left,League Left,League Joined,Market Valuation,...,Latitude_Left,Longitude_Left,Club_Name_TM_Joined,Club_Name_Wikidata_Joined,venueLabel_Joined,Latitude_Joined,Longitude_Joined,Transfer_x,Club Joined,Transfer_y


<div class="alert alert-warning" role="alert">

**Method** <br>
In following graph, we give a pictorial representation of transfers of size 50-75 million euros in the years 2011-2015 <br><br>

</div>

<div class="alert alert-success" role="alert">
    
**Inference**<br>
    In this five year period, the big transfers have not only increased but there is also an increase on the surplus front meaning that the clubs are spending higher amounts to acquire players and spending much above the player valuations as given by the transfer market. This could be a result of increasing competition among the clubs and higher demand for star players. This could be a result of increasing competition among the clubs and higher demand for star players. Importantly, we can see the entry of China in ‘Big’ Transfers category during this time.

    
</div>

In [46]:
club_left_lat = final_DataFrame_2011_2015.Latitude_Left
club_left_lon = final_DataFrame_2011_2015.Longitude_Left

club_join_lat = final_DataFrame_2011_2015.Latitude_Joined
club_join_lon = final_DataFrame_2011_2015.Longitude_Joined

fig = go.Figure()

fig.add_trace(
        go.Scattergeo(
            lon = merged_df_2011_2015.Longitude,
            lat = merged_df_2011_2015.Latitude,
            text = merged_df_2011_2015['Club Name Wikidata'],
            hoverinfo = 'text',
            mode = 'markers',
            marker = dict(
                size = np.log((merged_df_2011_2015['Transfer']+100)*100000)*0.9,
                opacity = 0.8,
                reversescale = False,
                autocolorscale = False,
                symbol = 'circle',
                line = dict(
                    width=1,
                    color='rgba(102, 102, 102)'),
            colorscale = 'Blues',
            cmin = merged_df_2011_2015['Surplus/Deficit'].min(),
            color = merged_df_2011_2015['Surplus/Deficit'],
            cmax = merged_df_2011_2015['Surplus/Deficit'].max(),
            colorbar_title="Surplus for the Club"
        )))

for i in range(len(final_DataFrame_2011_2015)):
    fig.add_trace(
        go.Scattergeo(
            lon = [final_DataFrame_2011_2015['Longitude_Left'][i], final_DataFrame_2011_2015['Longitude_Joined'][i]],
            lat = [final_DataFrame_2011_2015['Latitude_Left'][i], final_DataFrame_2011_2015['Latitude_Joined'][i]],
            mode = 'lines',
            hovertext = "Name: " + str(final_DataFrame_2011_2015['Name'][i]),
            line = dict(
                width = 1,
                color='darkgray')
        )
    )
    
fig.update_layout(
    title='Transfer Size: 50M to 75M (2011-2015)',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    mapbox_style="open-street-map"
    )

plot(fig, filename = 'Transfer_Size_50M_75M_2011_2015.html')

'Transfer_Size_50M_75M_2011_2015.html'

**CONTINUING WE NOW LOOK AT 2016-2017**

In [47]:
years = [2016, 2017]

final_DataFrame_2016_2017 = final_DataFrame_2[final_DataFrame_2.Year.isin(years)]

merged_df

Transfer_Value_2016_2017 = final_DataFrame_2016_2017.groupby('Club Joined').agg({'Transfer':'sum'}).reset_index()

Transfer_Surplus_2016_2017 = final_DataFrame_2016_2017.groupby('Club Joined').agg({'Surplus/Deficit':'sum'})

merged_df_2016_2017_x = pd.merge(Transfer_Surplus_2016_2017,Transfer_Value_2016_2017,
                    how='outer',
                    on = 'Club Joined')

merged_df_2016_2017 = pd.merge(merged_df,merged_df_2016_2017_x,
                    how='inner',
                    left_on = 'Club Name TM',
                    right_on = 'Club Joined')

merged_df_2016_2017 = merged_df_2016_2017.fillna(0)

merged_df_2016_2017['Transfer'] = merged_df_2016_2017['Transfer']

final_DataFrame_2016_2017 = pd.merge(final_DataFrame_2016_2017,Transfer_Value_2016_2017,
                    how='inner',
                    on = 'Club Joined')
final_DataFrame_2016_2017

,index,Date,Name,Age,Position,Nationality,Club Left,League Left,League Joined,Market Valuation,...,Latitude_Left,Longitude_Left,Club_Name_TM_Joined,Club_Name_Wikidata_Joined,venueLabel_Joined,Latitude_Joined,Longitude_Joined,Transfer_x,Club Joined,Transfer_y


<div class="alert alert-warning" role="alert">

**Method** <br>
In following graph, we give a pictorial representation of transfers of size 50-75 million euros in the years 2016-2017 <br><br>

</div>

<div class="alert alert-success" role="alert">
    
**Inference**<br>
    In the 2 year period between 2016-2017, the numbers don't seem to change much apart from the fact that surpluses have gone down a bit. Clubs have tried to put a check on their over-spendings. This could again be a result of better analysis and use of the transfer market data available. Again, we see a significant increase in China’s participation in the Big Transfers, underscoring their seriousness of becoming a powerhouse of world football.

    
</div>

In [48]:
club_left_lat = final_DataFrame_2016_2017.Latitude_Left
club_left_lon = final_DataFrame_2016_2017.Longitude_Left

club_join_lat = final_DataFrame_2016_2017.Latitude_Joined
club_join_lon = final_DataFrame_2016_2017.Longitude_Joined

fig = go.Figure()

fig.add_trace(
        go.Scattergeo(
            lon = merged_df_2016_2017.Longitude,
            lat = merged_df_2016_2017.Latitude,
            text = merged_df_2016_2017['Club Name Wikidata'],
            hoverinfo = 'text',
            mode = 'markers',
            marker = dict(
                size = np.log((merged_df_2016_2017['Transfer']+100)*100000)*0.9,
                opacity = 0.8,
                reversescale = False,
                autocolorscale = False,
                symbol = 'circle',
                line = dict(
                    width=1,
                    color='rgba(102, 102, 102)'),
            colorscale = 'Blues',
            cmin = merged_df_2016_2017['Surplus/Deficit'].min(),
            color = merged_df_2016_2017['Surplus/Deficit'],
            cmax = merged_df_2016_2017['Surplus/Deficit'].max(),
            colorbar_title="Surplus for the Club"
        )))

for i in range(len(final_DataFrame_2016_2017)):
    fig.add_trace(
        go.Scattergeo(
            lon = [final_DataFrame_2016_2017['Longitude_Left'][i], final_DataFrame_2016_2017['Longitude_Joined'][i]],
            lat = [final_DataFrame_2016_2017['Latitude_Left'][i], final_DataFrame_2016_2017['Latitude_Joined'][i]],
            mode = 'lines',
            hovertext = "Name: " + str(final_DataFrame_2016_2017['Name'][i]),
            line = dict(
                width = 1,
                color='darkgray')
        )
    )
    
fig.update_layout(
    title='Transfer Size: 50M to 75M (2016-2017)',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    mapbox_style="open-street-map"
    )

plot(fig, filename = 'Transfer_Size_50M_75M_2016_2017.html')

'Transfer_Size_50M_75M_2016_2017.html'

**FINALLY WE LOOK AT YEARS 2018-2019**

In [49]:
years = [2018, 2019]

final_DataFrame_2018_2019 = final_DataFrame_2[final_DataFrame_2.Year.isin(years)]

merged_df

Transfer_Value_2018_2019 = final_DataFrame_2018_2019.groupby('Club Joined').agg({'Transfer':'sum'}).reset_index()

Transfer_Surplus_2018_2019 = final_DataFrame_2018_2019.groupby('Club Joined').agg({'Surplus/Deficit':'sum'})

merged_df_2018_2019_x = pd.merge(Transfer_Surplus_2018_2019,Transfer_Value_2018_2019,
                    how='outer',
                    on = 'Club Joined')

merged_df_2018_2019 = pd.merge(merged_df,merged_df_2018_2019_x,
                    how='inner',
                    left_on = 'Club Name TM',
                    right_on = 'Club Joined')

merged_df_2018_2019 = merged_df_2018_2019.fillna(0)

merged_df_2018_2019['Transfer'] = merged_df_2018_2019['Transfer']

final_DataFrame_2018_2019 = pd.merge(final_DataFrame_2018_2019,Transfer_Value_2018_2019,
                    how='inner',
                    on = 'Club Joined')
final_DataFrame_2018_2019

,index,Date,Name,Age,Position,Nationality,Club Left,League Left,League Joined,Market Valuation,...,Latitude_Left,Longitude_Left,Club_Name_TM_Joined,Club_Name_Wikidata_Joined,venueLabel_Joined,Latitude_Joined,Longitude_Joined,Transfer_x,Club Joined,Transfer_y


<div class="alert alert-warning" role="alert">

**Method** <br>
In following graph, we give a pictorial representation of transfers of size 50-75 million euros in the years 2018-2019 <br><br>

</div>

<div class="alert alert-success" role="alert">
    
**Inference**<br>
    The number of tranfers seem to have reduced since 2017 and are mostly linked to the European leagues. China has continued and further solidified its mark on Football’s Transfer Market.

    
</div>

In [22]:
club_left_lat = final_DataFrame_2018_2019.Latitude_Left
club_left_lon = final_DataFrame_2018_2019.Longitude_Left

club_join_lat = final_DataFrame_2018_2019.Latitude_Joined
club_join_lon = final_DataFrame_2018_2019.Longitude_Joined

fig = go.Figure()

fig.add_trace(
        go.Scattergeo(
            lon = merged_df_2018_2019.Longitude,
            lat = merged_df_2018_2019.Latitude,
            text = merged_df_2018_2019['Club Name Wikidata'],
            hoverinfo = 'text',
            mode = 'markers',
            marker = dict(
                size = np.log((merged_df_2018_2019['Transfer']+100)*100000)*0.9,
                opacity = 0.8,
                reversescale = False,
                autocolorscale = False,
                symbol = 'circle',
                line = dict(
                    width=1,
                    color='rgba(102, 102, 102)'),
            colorscale = 'Blues',
            cmin = merged_df_2018_2019['Surplus/Deficit'].min(),
            color = merged_df_2018_2019['Surplus/Deficit'],
            cmax = merged_df_2018_2019['Surplus/Deficit'].max(),
            colorbar_title="Surplus for the Club"
        )))

for i in range(len(final_DataFrame_2018_2019)):
    fig.add_trace(
        go.Scattergeo(
            lon = [final_DataFrame_2018_2019['Longitude_Left'][i], final_DataFrame_2018_2019['Longitude_Joined'][i]],
            lat = [final_DataFrame_2018_2019['Latitude_Left'][i], final_DataFrame_2018_2019['Latitude_Joined'][i]],
            mode = 'lines',
            hovertext = "Name: " + str(final_DataFrame_2018_2019['Name'][i]),
            line = dict(
                width = 1,
                color='darkgray')
        )
    )
    
fig.update_layout(
    title='Transfer Size: 50M to 75M (2018-2019)',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    mapbox_style="open-street-map"
    )

plot(fig, filename = 'Transfer_Size_50M_75M_2018_2019.html')

'Transfer_Size_50M_75M_2018_2019.html'